# Machine Learning aided Record Linkage (and Data Fusion?) - a comparison between different ML methods

## Group Components
* **Francesco Porto**
* **Francesco Stranieri**
* **Mattia Vincenzi**

## Abstract
Record Linkage is the process of finding records in one or more datasets that refer to the same entity across different data sources. Traditionally, it is done by applying comparison rules between pairs of attributes from each dataset. In this project we investigate some possible Machine Learning applications to Data Linkage, and we compare them to the standard approach.

## Python Record Linkage Toolkit
Throughout the project, we make use of a Python library called "Python Record Linkage Toolkit", which provides a simple framework to facilitate the process of Record Linkage. In the context of this library, the Record Linkage process is dived into 5 steps:

* Preprocessing
* Indexing
* Comparison
* Classification
* Evaluation

Please refer to the documentation available at the following link for further information:

https://recordlinkage.readthedocs.io/en/latest/index.html

## Dataset description
We use the FEBRL dataset since it provides both the "golden links" for Record Linkage and a gold standard for Data Fusion. [TBC]

In [19]:
!pip install recordlinkage
import recordlinkage as rl

In [4]:
from recordlinkage.datasets import load_febrl4

'''
Generated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original), 
the originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) 
and dataset4b.csv (containing the 5000 duplicate records). 
These two data sets can be used for testing linkage procedures.
https://github.com/J535D165/FEBRL-fork-v0.4.2/tree/master/dsgen
'''

'\nGenerated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original), \nthe originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) \nand dataset4b.csv (containing the 5000 duplicate records). \nThese two data sets can be used for testing linkage procedures.\nhttps://github.com/J535D165/FEBRL-fork-v0.4.2/tree/master/dsgen\n'

In [5]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [6]:
# load datasets
print('Loading data...')
dfA, dfB, true_links = load_febrl4(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(dfB), 'records in dataset B')
print(len(true_links), 'links between dataset A and B')

Loading data...
5000 records in dataset A
5000 records in dataset B
5000 links between dataset A and B


In [7]:
dfA
dfA.dtypes

given_name       object
surname          object
street_number    object
address_1        object
address_2        object
suburb           object
postcode         object
state            object
date_of_birth    object
soc_sec_id       object
dtype: object

In [8]:
dfB

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884
...,...,...,...,...,...,...,...,...,...,...
rec-4495-dup-0,connor,belperio,15,NaN,NaN,ryde,2570,nsw,19170518,5394641
rec-4211-dup-0,daniel,maspn,9,derrington crescent,el pedro caravan park,sunnybank,4350,vic,19500705,5525378
rec-3131-dup-0,samuel,crofs,613,banjine street,kurrajong vlge,pengzin,2230,qld,19410531,4467228


In [9]:
dfA.loc['rec-1070-org']

given_name             michaela
surname                 neumann
street_number                 8
address_1        stanley street
address_2                 miami
suburb            winston hills
postcode                   4223
state                       nsw
date_of_birth          19151111
soc_sec_id              5304218
Name: rec-1070-org, dtype: object

In [10]:
dfB.loc['rec-1070-dup-0']

given_name             michafla
surname                 jakimow
street_number                 8
address_1        stanleykstreet
address_2                 miami
suburb            winstonbhills
postcode                   4223
state                       NaN
date_of_birth          19151111
soc_sec_id              5304218
Name: rec-1070-dup-0, dtype: object

## Indexing
Indexing is the process of creating all the possible links between the two datasets. In this specific example, we use a technique called **Blocking**, which groups together all the records that agree on AT LEAST one of the specified attributes. It is also capable of returning each link only once (and not twice) by only looking at the upper triangular matrix of matches.

In [11]:
from recordlinkage.index import Block
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('surname'))
# OR
indexer.add(Block('date_of_birth'))
# OR
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA, dfB)
print(len(candidate_links), 'candidate links between dataset A and B')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.83s - pairs: 87132/25000000 - rr: 0.99651
INFO:recordlinkage:indexing [1/?] - time: 0.83s - pairs_total: 87132/25000000 - rr_total: 0.99651
87132 candidate links between dataset A and B


In [12]:
candidate_links

MultiIndex([(  'rec-0-org',    'rec-0-dup-0'),
            (  'rec-0-org', 'rec-1505-dup-0'),
            (  'rec-0-org', 'rec-1636-dup-0'),
            (  'rec-0-org', 'rec-2074-dup-0'),
            (  'rec-0-org', 'rec-2683-dup-0'),
            (  'rec-0-org', 'rec-2724-dup-0'),
            (  'rec-0-org', 'rec-2894-dup-0'),
            (  'rec-1-org',    'rec-1-dup-0'),
            (  'rec-1-org', 'rec-1052-dup-0'),
            (  'rec-1-org', 'rec-2552-dup-0'),
            ...
            ('rec-999-org', 'rec-3685-dup-0'),
            ('rec-999-org',  'rec-370-dup-0'),
            ('rec-999-org', 'rec-3766-dup-0'),
            ('rec-999-org', 'rec-3862-dup-0'),
            ('rec-999-org', 'rec-3913-dup-0'),
            ('rec-999-org', 'rec-3940-dup-0'),
            ('rec-999-org', 'rec-4941-dup-0'),
            ('rec-999-org',  'rec-859-dup-0'),
            ('rec-999-org',  'rec-911-dup-0'),
            ('rec-999-org',  'rec-999-dup-0')],
           names=['rec_id_1', 'rec_id_2'], 

In [13]:
dfA.loc[candidate_links[0][0]]

given_name               rachael
surname                     dent
street_number                  1
address_1            knox street
address_2        lakewood estate
suburb                    byford
postcode                    4129
state                        vic
date_of_birth           19280722
soc_sec_id               1683994
Name: rec-0-org, dtype: object

In [14]:
dfB.loc[candidate_links[0][1]]

given_name               rachael
surname                     dent
street_number                  4
address_1            knox street
address_2        lakewood estate
suburb                    byford
postcode                    4129
state                        vic
date_of_birth           19280722
soc_sec_id               1683994
Name: rec-0-dup-0, dtype: object

In [15]:
dfB.loc[candidate_links[1][1]]

given_name                  emiily
surname                       dent
street_number                   27
address_1        gungurra crescent
address_2                 redlands
suburb                     whyalla
postcode                      3775
state                          nsw
date_of_birth             19960112
soc_sec_id                 9836985
Name: rec-1505-dup-0, dtype: object

## Comparison (aka "the classic approach")
**Comparison** refers to the process of evaluating all the possible links in order to figure out the best ones. In order to compare attributes, we need to specify (for each attribute):
* A **metric** to be used
* A **threshold** to decide under which circumstances the metric shall return true (= a match) or false (= not a match)

We decided not to use exact matches on strings as input errors (e.g. by an employee) might be common, therefore it would be too strict of a restriction.

In [23]:
from recordlinkage.compare import Exact, String
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(String('given_name', 'given_name', method='jarowinkler',
                    threshold=0.85, label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(String('date_of_birth', 'date_of_birth', method='jarowinkler',
                    threshold=0.85, label='date_of_birth'))
comparer.add(String('suburb', 'suburb', method='jarowinkler', 
                    threshold=0.85, label='suburb'))
comparer.add(String('state', 'state', method='jarowinkler', 
                    threshold=0.85, label='state'))
comparer.add(String('address_1', 'address_1', 
                    threshold=0.85, label='address_1'))
comparer.add(String('address_2', 'address_2', 
                    threshold=0.85, label='address_2'))
features = comparer.compute(candidate_links, dfA, dfB)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 25.43s - pairs: 87132
INFO:recordlinkage:comparing [1/?] - time: 25.43s - pairs_total: 87132
feature shape (87132, 7)


In [17]:
features

given_name  surname  date_of_birth  suburb  state  \
rec_id_1    rec_id_2                                                            
rec-0-org   rec-0-dup-0            1.0      1.0            1.0     1.0    1.0   
            rec-1505-dup-0         0.0      1.0            0.0     0.0    0.0   
            rec-1636-dup-0         0.0      1.0            0.0     0.0    0.0   
            rec-2074-dup-0         0.0      1.0            0.0     0.0    1.0   
            rec-2683-dup-0         0.0      1.0            0.0     0.0    0.0   
...                                ...      ...            ...     ...    ...   
rec-999-org rec-3940-dup-0         0.0      1.0            0.0     0.0    0.0   
            rec-4941-dup-0         0.0      1.0            0.0     0.0    0.0   
            rec-859-dup-0          0.0      1.0            0.0     0.0    0.0   
            rec-911-dup-0          0.0      1.0            0.0     0.0    1.0   
            rec-999-dup-0          1.0      1.0            1.0     1.0    1.0   

                            address_1  address_2  
rec_id_1    rec_id_2                              
rec-0-org   rec-0-dup-0           1.0        1.0  
            rec-1505-dup-0        0.0        0.0  
            rec-1636-dup-0        0.0        0.0  
            rec-2074-dup-0        0.0        0.0  
            rec-2683-dup-0        0.0        0.0  
...                               ...        ...  
rec-999-org rec-3940-dup-0        0.0        0.0  
            rec-4941-dup-0        0.0        0.0  
            rec-859-dup-0         0.0        0.0  
            rec-911-dup-0         0.0        0.0  
            rec-999-dup-0         1.0        1.0  

[87132 rows x 7 columns]

In [18]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

7.0     1725
6.0     1890
5.0     1004
4.0      310
3.0      995
2.0    20206
1.0    61002
dtype: int64

In [57]:
# no ML
matches = features[features.sum(axis=1) > 3]
print(len(matches))

4920


In [58]:
matches

,,given_name,surname,date_of_birth,suburb,state,address_1,address_2
rec_id_1,rec_id_2,,,,,,,
rec-0-org,rec-0-dup-0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
rec-1-org,rec-1-dup-0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
rec-10-org,rec-10-dup-0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
rec-100-org,rec-100-dup-0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
rec-1000-org,rec-1000-dup-0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
rec-995-org,rec-995-dup-0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
rec-996-org,rec-996-dup-0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
rec-997-org,rec-997-dup-0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [66]:
# return the confusion matrix
conf_noml = rl.confusion_matrix(true_links, matches, len(candidate_links))
print('confusion matrix')
print(conf_noml)

confusion matrix
[[ 4918    82]
 [    2 82130]]


In [67]:
# compute the F-score for this classification
fscore = rl.fscore(conf_noml)
print('fscore', fscore)
recall = rl.recall(true_links, matches)
print('recall', recall)
precision = rl.precision(true_links, matches)
print('precision', precision)

fscore 0.9915322580645163
recall 0.9836
precision 0.9995934959349594


## Logistic Regression Classifier

In [77]:
# use the Logistic Regression Classifier
# this classifier is equivalent to the deterministic record linkage approach
intercept = -11.0
coefficients = [1.5, 1.5, 8.0, 6.0, 2.5, 6.5, 5.0]

print('Deterministic classifier')
print('intercept', intercept)
print('coefficients', coefficients)

Deterministic classifier
intercept -11.0
coefficients [1.5, 1.5, 8.0, 6.0, 2.5, 6.5, 5.0]


In [78]:
logreg = rl.LogisticRegressionClassifier(
    coefficients=coefficients, intercept=intercept)
links = logreg.predict(features)

print(len(links), 'matches')

INFO:recordlinkage:Classification - predict matches and non-matches
5161 matches


In [79]:
# return the confusion matrix
conf_logreg = rl.confusion_matrix(true_links, links, len(candidate_links))
print('confusion matrix')
print(conf_logreg)

confusion matrix
[[ 4972    28]
 [  189 81943]]


In [80]:
# compute the F-score for this classification
fscore = rl.fscore(conf_logreg)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9786438342682806
recall 0.9944
precision 0.963379190079442


## Naive Bayes

In [81]:
# Initialise the NaiveBayesClassifier.
cl = rl.NaiveBayesClassifier()
cl.fit(features, true_links)

INFO:recordlinkage:Classification - start training NaiveBayesClassifier
INFO:recordlinkage:Classification - training computation time: ~0.29s


In [82]:
# Print the parameters that are trained (m, u and p). Note that the estimates
# are very good.
print("p probability P(Match):", cl.p)
print("m probabilities P(x_i=1|Match):", cl.m_probs)
print("u probabilities P(x_i=1|Non-Match):", cl.u_probs)
print("log weights of features:", cl.log_weights)
print("weights of features:", cl.weights)

p probability P(Match): 0.05726943028967549
m probabilities P(x_i=1|Match): {'given_name': {0.0: 0.20380762710189862, 1.0: 0.7961923728981007}, 'surname': {0.0: 0.1462925993469899, 1.0: 0.8537074006530102}, 'date_of_birth': {0.0: 0.0823647461978057, 1.0: 0.9176352538021937}, 'suburb': {0.0: 0.07474951604210353, 1.0: 0.9252504839578959}, 'state': {0.0: 0.05871745255641471, 1.0: 0.941282547443585}, 'address_1': {0.0: 0.1412825795077122, 1.0: 0.858717420492287}, 'address_2': {0.0: 0.3148296667402937, 1.0: 0.685170333259706}}
u probabilities P(x_i=1|Non-Match): {'given_name': {0.0: 0.9940712412795613, 1.0: 0.0059287587204383645}, 'surname': {0.0: 0.0077305166474385435, 1.0: 0.9922694833525615}, 'date_of_birth': {0.0: 0.9820432897128304, 1.0: 0.01795671028716927}, 'suburb': {0.0: 0.9978451936942235, 1.0: 0.0021548063057758365}, 'state': {0.0: 0.7818777232551495, 1.0: 0.21812227674485082}, 'address_1': {0.0: 0.9993912967802312, 1.0: 0.0006087032197689292}, 'address_2': {0.0: 0.99948868910060

In [83]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches
Predicted number of links: 4985


In [84]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[ 4977    23]
 [    8 82124]]


In [85]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9968953430145219
recall 0.9954
precision 0.9983951855566701


In [86]:
# Predict the match probability for each pair in the dataset.
probs = cl.prob(features)
probs

INFO:recordlinkage:Classification - compute probabilities


rec_id_1     rec_id_2      
rec-0-org    rec-0-dup-0       1.000000e+00
             rec-1505-dup-0    2.251400e-07
             rec-1636-dup-0    2.251400e-07
             rec-2074-dup-0    1.293715e-05
             rec-2683-dup-0    2.251400e-07
                                   ...     
rec-999-org  rec-3940-dup-0    2.251400e-07
             rec-4941-dup-0    2.251400e-07
             rec-859-dup-0     2.251400e-07
             rec-911-dup-0     1.293715e-05
             rec-999-dup-0     1.000000e+00
Length: 87132, dtype: float64

## Expectation-Conditional Maximisation

In [89]:
'''
Example: Unsupervised learning with the ECM algorithm.
Train data is often hard to collect in record linkage or data matching
problems. The Expectation-Conditional Maximisation (ECM) algorithm is the most
well known algorithm for unsupervised data matching. The algorithm preforms
relatively well compared to supervised methods.
'''
import numpy as np

In [90]:
# Initialise the Expectation-Conditional Maximisation classifier.
cl = rl.ECMClassifier()
cl.fit(features)

INFO:recordlinkage:Classification - start training ECMClassifier
INFO:recordlinkage:Classification - training computation time: ~0.27s


INFO:recordlinkage:Classification - training computation time: ~0.27s


In [91]:
# Print the parameters that are trained (m, u and p). Note that the estimates
# are very good.
print("p probability P(Match):", cl.p)
print("m probabilities P(x_i=1|Match):", cl.m_probs)
print("u probabilities P(x_i=1|Non-Match):", cl.u_probs)
print("log weights of features:", cl.log_weights)
print("weights of features:", cl.weights)

p probability P(Match): 0.0576117927528846
m probabilities P(x_i=1|Match): {'given_name': {0.0: 0.20887754384968493, 1.0: 0.7911224561503161}, 'surname': {0.0: 0.151805994706334, 1.0: 0.8481940052936664}, 'date_of_birth': {0.0: 0.08170046345190825, 1.0: 0.918299536548092}, 'suburb': {0.0: 0.07994263723737696, 1.0: 0.9200573627626235}, 'state': {0.0: 0.0594790279590614, 1.0: 0.9405209720409399}, 'address_1': {0.0: 0.146509414260776, 1.0: 0.8534905857392252}, 'address_2': {0.0: 0.31855780722347266, 1.0: 0.6814421927765274}}
u probabilities P(x_i=1|Non-Match): {'given_name': {0.0: 0.9940483951307649, 1.0: 0.005951604869236727}, 'surname': {0.0: 0.007343121107290269, 1.0: 0.992656878892712}, 'date_of_birth': {0.0: 0.9824107463801766, 1.0: 0.01758925361982419}, 'suburb': {0.0: 0.9978630721592132, 1.0: 0.0021369278407886807}, 'state': {0.0: 0.7820938834743134, 1.0: 0.2179061165256877}, 'address_1': {0.0: 0.9993835051043114, 1.0: 0.0006164948956893574}, 'address_2': {0.0: 0.9995095058020591, 

In [92]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches


INFO:recordlinkage:Classification - predict matches and non-matches


Predicted number of links: 4985


In [93]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[ 4977    23]
 [    8 82124]]


In [94]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9968953430145219
recall 0.9954
precision 0.9983951855566701


In [95]:
# Predict the match probability for each pair in the dataset.
probs = cl.prob(features)
print(probs)

INFO:recordlinkage:Classification - compute probabilities


INFO:recordlinkage:Classification - compute probabilities


rec_id_1     rec_id_2      
rec-0-org    rec-0-dup-0       1.000000e+00
             rec-1505-dup-0    2.598605e-07
             rec-1636-dup-0    2.598605e-07
             rec-2074-dup-0    1.474783e-05
             rec-2683-dup-0    2.598605e-07
                                   ...     
rec-999-org  rec-3940-dup-0    2.598605e-07
             rec-4941-dup-0    2.598605e-07
             rec-859-dup-0     2.598605e-07
             rec-911-dup-0     1.474783e-05
             rec-999-dup-0     1.000000e+00
Length: 87132, dtype: float64
